In [0]:
#Load Data
options = {
    "header": "true",
    "inferSchema": "true",
    "delimiter": ";"
} 
dim_product =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_product/")
fact_int_sales =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/fact_int_sales/")
dim_sales_terr =  spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_sales_terr/")
dim_curreny = spark.read.format("csv").options(**options).load("dbfs:/FileStore/tables_new/dim_curreny/")  

In [0]:
# Installing Delta Lake 
#%pip install delta-spark

In [0]:
# We will convert the Fact Sales table into a delta table
fact_int_sales.write.format("delta").save("/mnt/delta/fact_sales_delta")

In [0]:
display(dbutils.fs.ls("/mnt/delta/fact_sales_delta"))

path,name,size,modificationTime
dbfs:/mnt/delta/fact_sales_delta/_delta_log/,_delta_log/,0,0
dbfs:/mnt/delta/fact_sales_delta/part-00000-ea046cba-5136-4843-8236-69585a22f59a-c000.snappy.parquet,part-00000-ea046cba-5136-4843-8236-69585a22f59a-c000.snappy.parquet,526011,1746809348000
dbfs:/mnt/delta/fact_sales_delta/part-00001-7947d1f2-5608-4bdb-9877-1cefaa9e7725-c000.snappy.parquet,part-00001-7947d1f2-5608-4bdb-9877-1cefaa9e7725-c000.snappy.parquet,394357,1746809348000


## Time Travel

In [0]:
 fact_sales_delta = spark.read.format("delta").load("/mnt/delta/fact_sales_delta")
display(fact_sales_delta.limit(10))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate
310,20101229,20110110,20110105,21768,1,19,6,SO43697,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
336,20101229,20110110,20110105,14501,1,100,4,SO43700,1,null,null,413.15,413.15,699.1,55.93,174.775,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
311,20101230,20110111,20110106,27645,1,100,4,SO43702,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
310,20101230,20110111,20110106,16624,1,6,9,SO43703,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-30,2011-01-11,2011-01-06
351,20101230,20110111,20110106,11005,1,6,9,SO43704,1,3374.99,3374.99,1898.09,1898.09,3374.99,270.0,843.748,2010-12-30,2011-01-11,2011-01-06
344,20101230,20110111,20110106,11011,1,6,9,SO43705,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-30,2011-01-11,2011-01-06
312,20101231,20110112,20110107,27621,1,100,4,SO43706,1,3578.27,3578.27,2171.29,2171.29,3578.27,286.26,894.568,2010-12-31,2011-01-12,2011-01-07


In [0]:
# Delete a key
from delta.tables import DeltaTable

# Load as DeltaTable (not DataFrame)
sales_delta_table = DeltaTable.forPath(spark, "/mnt/delta/fact_sales_delta")
 
sales_delta_table.delete("ProductKey = 346")  

In [0]:
# We can select directly from the delta table  
spark.sql("SELECT * FROM delta.`/mnt/delta/fact_sales_delta` where ProductKey = 346  LIMIT 10").show()

+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+---------+--------------+-------------------+----------------+-----------+------+-------+---------+-------+--------+
|ProductKey|OrderDateKey|DueDateKey|ShipDateKey|CustomerKey|PromotionKey|CurrencyKey|SalesTerritoryKey|SalesOrderNumber|SalesOrderLineNumber|UnitPrice|ExtendedAmount|ProductStandardCost|TotalProductCost|SalesAmount|TaxAmt|Freight|OrderDate|DueDate|ShipDate|
+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+---------+--------------+-------------------+----------------+-----------+------+-------+---------+-------+--------+
+----------+------------+----------+-----------+-----------+------------+-----------+-----------------+----------------+--------------------+---------+--------------+-------------------+----------------+-----------+------+----

In [0]:
# The deletion created a new version of the file without the deleted ProductKey
display(sales_delta_table.history()) 

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-05-09T16:49:36.000+0000,4338314464403418,yazanziyad98@gmail.com,DELETE,"Map(predicate -> [""(ProductKey#22393 = 346)""])",null,List(4259928558689402),0509-155856-anqi0nes,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 526011, numCopiedRows -> 32366, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 5749, numDeletedRows -> 49, scanTimeMs -> 2346, numAddedFiles -> 1, numAddedBytes -> 524967, rewriteTimeMs -> 3373)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-05-09T16:49:09.000+0000,4338314464403418,yazanziyad98@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(4259928558689402),0509-155856-anqi0nes,null,WriteSerializable,true,"Map(numFiles -> 2, numOutputRows -> 60398, numOutputBytes -> 920368)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
# This returns the latest vesion
display(
  spark.read.format("delta")
    .load("/mnt/delta/fact_sales_delta")
    .filter("ProductKey = 346"))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate


In [0]:
#This will return the first version
display( spark.read.format("delta").option("versionAsOf", 0).load("/mnt/delta/fact_sales_delta") .filter("ProductKey = 346"))

ProductKey,OrderDateKey,DueDateKey,ShipDateKey,CustomerKey,PromotionKey,CurrencyKey,SalesTerritoryKey,SalesOrderNumber,SalesOrderLineNumber,UnitPrice,ExtendedAmount,ProductStandardCost,TotalProductCost,SalesAmount,TaxAmt,Freight,OrderDate,DueDate,ShipDate
346,20101229,20110110,20110105,28389,1,39,7,SO43698,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,25863,1,100,1,SO43699,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20101229,20110110,20110105,11003,1,6,9,SO43701,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2010-12-29,2011-01-10,2011-01-05
346,20110106,20110118,20110113,11238,1,98,10,SO43729,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-06,2011-01-18,2011-01-13
346,20110106,20110118,20110113,25861,1,100,4,SO43730,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-06,2011-01-18,2011-01-13
346,20110107,20110119,20110114,11002,1,6,9,SO43736,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-07,2011-01-19,2011-01-14
346,20110114,20110126,20110121,11010,1,6,9,SO43765,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-14,2011-01-26,2011-01-21
346,20110123,20110204,20110130,11026,1,6,9,SO43811,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-23,2011-02-04,2011-01-30
346,20110124,20110205,20110131,11006,1,6,9,SO43819,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-01-24,2011-02-05,2011-01-31
346,20110201,20110213,20110208,11034,1,6,9,SO43934,1,3399.99,3399.99,1912.15,1912.15,3399.99,272.0,849.998,2011-02-01,2011-02-13,2011-02-08


## Schema Evolution

In [0]:
# Create a new version of the table as a datafrom  with one column renamed
Updated_sales_delta = sales_delta_table.toDF().withColumnRenamed("SalesOrderLineNumber", "Sales_Order_Line")


In [0]:
# Check the new name
Updated_sales_delta.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- OrderDateKey: integer (nullable = true)
 |-- DueDateKey: integer (nullable = true)
 |-- ShipDateKey: integer (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- PromotionKey: integer (nullable = true)
 |-- CurrencyKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- Sales_Order_Line: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- ExtendedAmount: double (nullable = true)
 |-- ProductStandardCost: double (nullable = true)
 |-- TotalProductCost: double (nullable = true)
 |-- SalesAmount: double (nullable = true)
 |-- TaxAmt: double (nullable = true)
 |-- Freight: double (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- DueDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)



In [0]:
# Overwrite the existing delta table scheme to see how does the delta tables handles it
# This allowes us to overwrite existing delta tables without a shcema mishmatch error due to renaming a column 
Updated_sales_delta.write.format("delta") \
  .mode("overwrite").option("overwriteSchema", "true").save("/mnt/delta/fact_sales_delta")

In [0]:
# Check the updated delta table 
Updated_sales_delta =  DeltaTable.forPath(spark, "/mnt/delta/fact_sales_delta")
Updated_sales_delta.toDF().printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- OrderDateKey: integer (nullable = true)
 |-- DueDateKey: integer (nullable = true)
 |-- ShipDateKey: integer (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- PromotionKey: integer (nullable = true)
 |-- CurrencyKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- Sales_Order_Line: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- ExtendedAmount: double (nullable = true)
 |-- ProductStandardCost: double (nullable = true)
 |-- TotalProductCost: double (nullable = true)
 |-- SalesAmount: double (nullable = true)
 |-- TaxAmt: double (nullable = true)
 |-- Freight: double (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- DueDate: date (nullable = true)
 |-- ShipDate: date (nullable = true)



In [0]:
# Now we can check the  history to see that 
history_df = Updated_sales_delta.history()
display(history_df.select("version", "timestamp", "operation", "operationParameters"))

version,timestamp,operation,operationParameters
2,2025-05-09T16:50:21.000+0000,WRITE,"Map(mode -> Overwrite, partitionBy -> [])"
1,2025-05-09T16:49:36.000+0000,DELETE,"Map(predicate -> [""(ProductKey#22393 = 346)""])"
0,2025-05-09T16:49:09.000+0000,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])"
